## A take on a better approach

* Encode/transform all `categorical variables` of your data (on the `entire dataset`, this ensures categorical variables are encoded the same across training/test sets, if you can't do this, make sure the training and test sets have the same column names).

* Split your data (into train/test).

* Fill the training set and test set `numerical values separately`.

* Don’t use numerical data from the future (test set) to fill data from the past (training set).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [2]:
df = pd.read_csv("data/TrainAndValid.csv", low_memory=False,  parse_dates=["saledate"])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 12  fi

In [4]:
df.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

### Feature Engineering

In [3]:
features = {"Year", "Month", "Day", "DayOfWeek", "DayOfYear"}

def add_features(df, features_list, column_name, prefix):
    """
    Extract and add date features from the column_name to the Dataframe
    """
    for feature in features_list:
        df[f"{prefix}_{feature}"] =  getattr(df[column_name].dt, feature.lower()) 
        
    df.drop(column_name, axis=1, inplace=True)
    
    return df

In [4]:
df = add_features(df, features, "saledate", "Sale")
df.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000.0,57000.0,10000.0,38500.0,11000.0
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3.0,3.0,3.0,3.0,3.0
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68.0,4640.0,2838.0,3486.0,722.0
UsageBand,Low,Low,High,High,Medium
fiModelDesc,521D,950FII,226,PC120-6E,S175


In [5]:
def preprocess_non_numeric(df):
    """
    Perfoms encoding on non-numeric data in a Dataframe and returns it
    """
    for label, content in df.items():
        if not pd.api.types.is_numeric_dtype(content):
            # add a column to indicate that it was originally missing or not
            df[label+ "_is_missing"] = content.isna()
            # convert the column to contain category codes data
            df[label] = pd.Categorical(df[label]).codes + 1
        
    return df 

In [6]:
def preprocess_numeric(df):
    """
    Fills in missing numeric data in a Dataframe and returns it
    """
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if content.isna().sum():
                df[f"{label}_is_missing"] = content.isna()
                content.fillna(content.median(), inplace=True)
    
    return df

In [7]:
# Encode/transform all categorical variables of your data on the entire dataset

df = preprocess_non_numeric(df)

df.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000.0,57000.0,10000.0,38500.0,11000.0
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
...,...,...,...,...,...
Backhoe_Mounting_is_missing,True,True,True,True,True
Blade_Type_is_missing,True,True,True,True,True
Travel_Controls_is_missing,True,True,True,True,True
Differential_Type_is_missing,False,False,True,True,True


In [8]:
df["auctioneerID"].isna().sum()

20136

In [9]:
# Split your data (into train/test)

# Train set: rows that have a saleYear < 2012
df_train = df[df["Sale_Year"] != 2012]
df_val =  df[df["Sale_Year"] == 2012]

X_train = df_train.drop("SalePrice", axis=1)
y_train = df_train["SalePrice"]

X_val = df_val.drop("SalePrice", axis=1)
y_val = df_val["SalePrice"]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((401125, 100), (401125,), (11573, 100), (11573,))

In [10]:
# Fill the training set and test set numerical values separately
X_train = preprocess_numeric(X_train)
X_val = preprocess_numeric(X_val)

In [11]:
X_val["auctioneerID"].isna().sum()

0

In [12]:
X_train.shape, X_val.shape

((401125, 102), (11573, 101))

seems there is a missing column in X_val, lets find out which:

In [13]:
set(X_train.columns) - set(X_val.columns)

{'auctioneerID_is_missing'}

Lets add the missing column to X_val and set all the rows to False

In [14]:
X_val["auctioneerID_is_missing"] = False

In [15]:
X_train.shape, X_val.shape

((401125, 102), (11573, 102))

In [18]:
X_cols = list(X_train.columns)
X_cols

['SalesID',
 'MachineID',
 'ModelID',
 'datasource',
 'auctioneerID',
 'YearMade',
 'MachineHoursCurrentMeter',
 'UsageBand',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'ProductSize',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Drive_System',
 'Enclosure',
 'Forks',
 'Pad_Type',
 'Ride_Control',
 'Stick',
 'Transmission',
 'Turbocharged',
 'Blade_Extension',
 'Blade_Width',
 'Enclosure_Type',
 'Engine_Horsepower',
 'Hydraulics',
 'Pushblock',
 'Ripper',
 'Scarifier',
 'Tip_Control',
 'Tire_Size',
 'Coupler',
 'Coupler_System',
 'Grouser_Tracks',
 'Hydraulics_Flow',
 'Track_Type',
 'Undercarriage_Pad_Width',
 'Stick_Length',
 'Thumb',
 'Pattern_Changer',
 'Grouser_Type',
 'Backhoe_Mounting',
 'Blade_Type',
 'Travel_Controls',
 'Differential_Type',
 'Steering_Controls',
 'Sale_DayOfWeek',
 'Sale_DayOfYear',
 'Sale_Year',
 'Sale_Month',
 'Sale_Day',
 'UsageBand_is_missing',
 'fiModelDesc_is_missing',
 'fiBaseMo

In [19]:
# reorder the columns to match those of X_train

X_val = X_val.reindex(columns=X_cols)
X_val.head().T

,401125,401126,401127,401128,401129
SalesID,1222837,1222839,1222841,1222843,1222845
MachineID,902859,1048320,999308,1062425,1032841
ModelID,1376,36526,4587,1954,4701
datasource,121,121,121,121,121
auctioneerID,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...
Travel_Controls_is_missing,True,True,True,True,True
Differential_Type_is_missing,True,True,True,True,False
Steering_Controls_is_missing,True,True,True,True,False
auctioneerID_is_missing,False,False,False,False,False


In [20]:
X_train.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...
Travel_Controls_is_missing,True,True,True,True,True
Differential_Type_is_missing,False,False,True,True,True
Steering_Controls_is_missing,False,False,True,True,True
auctioneerID_is_missing,False,False,False,False,False


Now the sets have the same columns and order.

### Modeling

In [21]:
%%time

from sklearn.ensemble import RandomForestRegressor

# Instantiate the model
base_model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

# Fit the model
base_model.fit(X=X_train, y=y_train)

CPU times: total: 16min 33s
Wall time: 1min 30s


RandomForestRegressor(n_jobs=-1, random_state=42)

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score

def rmsle(y_true, y_pred):
    """
    Calculates root mean squared log error between actual and predicted labels
    """
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [23]:
def print_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Validation MAE": mean_absolute_error(y_val, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Validation RMSLE": rmsle(y_val, val_preds),
              "Training R^2": r2_score(y_train, train_preds),
              "Validation R^2": r2_score(y_val, val_preds)}
    print(scores)

In [24]:
base_scores = print_scores(base_model)

{'Training MAE': 1578.3270456840144, 'Validation MAE': 6121.3158671044675, 'Training RMSLE': 0.08415683199700703, 'Validation RMSLE': 0.2529128230708653, 'Training R^2': 0.9875241018499986, 'Validation R^2': 0.8731770299952705}


In [25]:
%%time

ideal_model = RandomForestRegressor(n_estimators=40,
                                    min_samples_leaf=1,
                                    min_samples_split=14,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42)

ideal_model.fit(X_train, y_train)

CPU times: total: 3min 13s
Wall time: 19 s


RandomForestRegressor(max_features=0.5, min_samples_split=14, n_estimators=40,
                      n_jobs=-1, random_state=42)

In [26]:
ideal_model_scores = print_scores(ideal_model)

{'Training MAE': 2957.1049886461487, 'Validation MAE': 5971.25839765751, 'Training RMSLE': 0.14474903379312803, 'Validation RMSLE': 0.24660734119669697, 'Training R^2': 0.9588012726736933, 'Validation R^2': 0.8811956891328139}


### Hyperparameter tuning

In [27]:
%%time

# let's use RandomizedSearchCV to find the best params
from sklearn.model_selection import RandomizedSearchCV

# create param_grid
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

# instantiate model
rs_model = RandomizedSearchCV(estimator=RandomForestRegressor(),
                              n_jobs=-1,
                              random_state=42,
                              param_distributions=rf_grid,
                              cv=5,
                              n_iter=100,
                              verbose=True)

rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: total: 16.7 s
Wall time: 5min 8s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                                        'n_estimators': array([10, 20, 30, 40, 50, 60, 70, 80, 90])},
                   random_state=42, verbose=True)

In [28]:
rs_model.best_params_

{'n_estimators': 90,
 'min_samples_split': 18,
 'min_samples_leaf': 3,
 'max_samples': 10000,
 'max_features': 0.5,
 'max_depth': None}

In [29]:
%%time

best_model = RandomForestRegressor(n_jobs=-1,
                                   random_state=42,
                                   n_estimators=90,
                                   min_samples_split=18,
                                   min_samples_leaf=3,
                                   max_samples=None,
                                   max_features=0.5,
                                   max_depth=None)

best_model.fit(X_train, y_train)

CPU times: total: 6min 36s
Wall time: 35.5 s


RandomForestRegressor(max_features=0.5, min_samples_leaf=3,
                      min_samples_split=18, n_estimators=90, n_jobs=-1,
                      random_state=42)

In [30]:
best_model_scores = print_scores(best_model)

{'Training MAE': 3350.3820077224723, 'Validation MAE': 5930.837773848372, 'Training RMSLE': 0.16314637502328522, 'Validation RMSLE': 0.2437659754796325, 'Training R^2': 0.9459331976619616, 'Validation R^2': 0.8811232022028774}
